In [1]:
!nvidia-smi

Tue Oct  9 11:48:04 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:04:00.0 Off |                  N/A |
| 23%   25C    P8    16W / 250W |      0MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:06:00.0 Off |                  N/A |
| 23%   25C    P8    16W / 250W |      0MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0, 1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0, 1


In [3]:
import glob
import pandas 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from sklearn.cross_validation import train_test_split
import random

import os
from tensorflow.python.client import device_lib
print (device_lib.list_local_devices())

/home/jimmy15923/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8981302488743614517
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10917275239
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 351386037889267243
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10917275239
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 8459818452401297812
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1"
]


In [ ]:
import glob
import pandas 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from sklearn.cross_validation import train_test_split
import random
from keras import backend as K
import keras
from keras.utils import to_categorical
from keras.utils.training_utils import multi_gpu_model

import sys
sys.path.append('keras-deeplab-v3-plus/')
from deeplab_v3_plus.model import *

import cv2
import numpy as np
import os
import random
from skimage import io
from skimage.transform import resize
from skimage import img_as_bool

def read_data_and_split(split_seed, train_ratio, is_normalize=True):
    """read data into np array, normalize it and train test split
    split_seed: set seed for same train test split
    train_ratio: ratio of training set. range from 0 to 1
    is_normalize: True for normalizr to -1 to 1
    
    return np array with x_train, x_test, y_train, y_test
    """
    
    idx = next(os.walk('train/'))[1]
    # remove two file with different size between image & mask
    idx.remove("S2016-30816_9_0")
    idx.remove("S2016-30816_9_1")
    
    # set seed
    random.seed(split_seed)
    random.shuffle(idx)
    
    train_idx, test_idx = idx[:int(len(idx)*train_ratio)], idx[int(len(idx)*train_ratio):]

    x_train = np.array([cv2.imread('train/{}/image/{}_slide.jpg'.format(x, x))[...,::-1]\
                    for x in train_idx], dtype="float32")
    x_test = np.array([cv2.imread('train/{}/image/{}_slide.jpg'.format(x, x))[...,::-1]\
                       for x in test_idx], dtype="float32")
    
    if is_normalize:
        x_train = (x_train / 127.5) - 1
        x_test = (x_test / 127.5) - 1
        
    y_train = np.array([cv2.imread('/data/jimmy15923/cg_kidney_seg/train/{}/mask/{}_mask.jpg'.format(x, x))[..., 0]\
                    for x in train_idx])
    
    y_test = np.array([cv2.imread('/data/jimmy15923/cg_kidney_seg/train/{}/mask/{}_mask.jpg'.format(x, x))[..., 0]\
                        for x in test_idx])
    
    y_train = img_as_bool(y_train)
    y_test = img_as_bool(y_test)
    
    return x_train, x_test, y_train, y_test

def cv2_resize(array):
    return np.array([resize(x, (500,500)) for x in array])

x_train, x_test, y_train, y_test = read_data_and_split(split_seed=7, train_ratio=0.8, is_normalize=True)

x_train = cv2_resize(x_train)
x_test = cv2_resize(x_test)
y_train = cv2_resize(y_train)
y_test = cv2_resize(y_test)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

y_train_inv = np.where(y_train, 0, 1)
y_train_ = np.zeros(shape=(len(y_train), 500,500, 2))
y_train_[:,:,:,0] = y_train
y_train_[:,:,:,1] = y_train_inv

y_test_inv = np.where(y_test, 0, 1)
y_test_ = np.zeros(shape=(len(y_test), 500,500, 2))
y_test_[:,:,:,0] = y_test
y_test_[:,:,:,1] = y_test_inv

def data_gen(x_train, y_train, bz, augmentation=None):
    i = 0
    from sklearn.utils import shuffle
    while True:
#         if i == len(y_train):
#             i = 0
#             x_train, y_train = shuffle(x_train, y_train)
            
#         x_, y_ = x_train[i*bz:(i+1)*bz], y_train[i*bz:(i+1)*bz]
        img_idx = np.random.choice(range(len(y_train)), bz, replace=False)
        

        yield x_train[img_idx], y_train[img_idx]
        
# def val_gen(x_test, y_test, crop_size=500, stride=500):
#     i = 0
#     while True:
#         x = []
#         y = []
#         for x_start in range(0, crop_size+1, stride):
#             for y_start in range(0, crop_size+1, stride):
#                 x_crop = x_test[i][x_start:(x_start+crop_size), y_start:(y_start+crop_size), :]
#                 y_crop = y_test[i][x_start:(x_start+crop_size), y_start:(y_start+crop_size), :]
#                 x.append(x_crop)
#                 y.append(y_crop)
#         i+=1
#         yield np.array(x), np.array(y)
#         if i == len(y_test):
#             i=0

crop_size = 500
model = Deeplabv3(input_shape=(crop_size, crop_size, 3), classes=2, OS=8)
logits = model.output
output = keras.layers.Activation("softmax")(logits)
model = Model(model.input, output)

def dice_coef_loss(y_true, y_pred, smooth = 1):
    def dice_coef_fix(y_true, y_pred):
        intersection = K.sum(K.abs(y_true * y_pred), axis = -1)
        iou = (2. * intersection + smooth) / (K.sum(K.square(y_true), -1) + K.sum(K.square(y_pred),-1) + smooth)
        return iou
    loss = 1 - dice_coef_fix(y_true, y_pred)
    return loss

model_gpu = multi_gpu_model(model, gpus=2)

model_gpu.compile(optimizer=keras.optimizers.SGD(lr=1e-4, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy')

early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=12, verbose=1)
check = keras.callbacks.ModelCheckpoint(monitor="val_loss",
                                        filepath="/data/jimmy15923/cg_kidney_seg/test_resize.h5",
                                        verbose=1, save_best_only=True, save_weights_only=True)

reduce = keras.callbacks.ReduceLROnPlateau(patience=3)


res = model_gpu.fit_generator(data_gen(x_train, y_train_, 12),
                    steps_per_epoch=200,
                    epochs=1000, 
                    validation_data=(x_test, y_test_),
                    callbacks=[early, check, reduce]
                   )

/home/jimmy15923/.conda/envs/tf18_keras/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from uint8 to bool
  .format(dtypeobj_in, dtypeobj_out))
/home/jimmy15923/.conda/envs/tf18_keras/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jimmy15923/.conda/envs/tf18_keras/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(733, 500, 500, 3)
(184, 500, 500, 3)
(733, 500, 500)
(184, 500, 500)
Epoch 1/1000
200/200 [==============================] - 173s 865ms/step - loss: 0.2928 - val_loss: 0.1330

Epoch 00001: val_loss improved from inf to 0.13305, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 2/1000
200/200 [==============================] - 153s 765ms/step - loss: 0.1435 - val_loss: 0.1179

Epoch 00002: val_loss improved from 0.13305 to 0.11786, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 3/1000
200/200 [==============================] - 153s 765ms/step - loss: 0.1230 - val_loss: 0.1112

Epoch 00003: val_loss improved from 0.11786 to 0.11121, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 4/1000
200/200 [==============================] - 154s 770ms/step - loss: 0.1164 - val_loss: 0.1063

Epoch 00004: val_loss improved from 0.11121 to 0.10634, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 5/1000
200/200 [============

Epoch 37/1000
200/200 [==============================] - 152s 758ms/step - loss: 0.0481 - val_loss: 0.0496

Epoch 00037: val_loss improved from 0.04996 to 0.04958, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 38/1000
200/200 [==============================] - 153s 763ms/step - loss: 0.0461 - val_loss: 0.0492

Epoch 00038: val_loss improved from 0.04958 to 0.04919, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 39/1000
200/200 [==============================] - 154s 768ms/step - loss: 0.0468 - val_loss: 0.0488

Epoch 00039: val_loss improved from 0.04919 to 0.04878, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 40/1000
200/200 [==============================] - 152s 758ms/step - loss: 0.0466 - val_loss: 0.0483

Epoch 00040: val_loss improved from 0.04878 to 0.04825, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 41/1000
200/200 [==============================] - 152s 762ms/step - loss: 0.0453 - val_lo

200/200 [==============================] - 146s 730ms/step - loss: 0.0351 - val_loss: 0.0404

Epoch 00073: val_loss improved from 0.04055 to 0.04043, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 74/1000
200/200 [==============================] - 146s 732ms/step - loss: 0.0355 - val_loss: 0.0403

Epoch 00074: val_loss improved from 0.04043 to 0.04030, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 75/1000
200/200 [==============================] - 147s 733ms/step - loss: 0.0350 - val_loss: 0.0402

Epoch 00075: val_loss improved from 0.04030 to 0.04016, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 76/1000
200/200 [==============================] - 149s 743ms/step - loss: 0.0352 - val_loss: 0.0400

Epoch 00076: val_loss improved from 0.04016 to 0.04000, saving model to /data/jimmy15923/cg_kidney_seg/test_resize.h5
Epoch 77/1000
200/200 [==============================] - 147s 733ms/step - loss: 0.0339 - val_loss: 0.0399

Ep

In [4]:
import keras

In [7]:
model = Deeplabv3(backbone="xception", input_shape=(500,500, 3), classes=2, OS=8)
logits = model.output
output = keras.layers.Activation("softmax")(logits)
model = Model(model.input, output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 500, 500, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 250, 250, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 250, 250, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_6 (Activation)       (None, 250, 250, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [1]:
import sys
sys.path.append('keras-deeplab-v3-plus/')
from deeplab_v3_plus.model import *

/home/jimmy15923/.conda/envs/tf18_keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
### code for prepare raw data
# image_list = glob.glob("/mnt/dataset/mask_roi/*.jpg")

# !rm -r /data/jimmy15923/cg_kidney_seg/

# !mkdir /data/jimmy15923/cg_kidney_seg

# for idx in set(all_idx):
#     os.makedirs("/data/jimmy15923/cg_kidney_seg/{}/mask/".format(idx))
#     os.makedirs("/data/jimmy15923/cg_kidney_seg/{}/image/".format(idx))

# for img in image_list:
#     name = os.path.basename(img)[:-4]
#     if "2018" in name:
#         new = name.split(" ")[2]
#     else:
#         new = name
#     if "mask" in new:
#         idx = new.split("_")[0] + "_" + new.split("_")[3] + "_" + new.split("_")[4] 
#     else:
#         idx = new.split("_")[0] + "_" + new.split("_")[2] + "_" + new.split("_")[3]  

#     if "mask" in new:
#         shutil.copy2(img, '/data/jimmy15923/cg_kidney_seg/{}/mask/{}_mask.jpg'.format(idx, idx))
#     else:
#         shutil.copy2(img, '/data/jimmy15923/cg_kidney_seg/{}/image/{}_slide.jpg'.format(idx, idx))

In [ ]:
import cv2
import numpy as np
import os
import random

def read_data_and_split(split_seed, train_ratio, is_normalize=True):
    """read data into np array, normalize it and train test split
    split_seed: set seed for same train test split
    train_ratio: ratio of training set. range from 0 to 1
    is_normalize: True for normalizr to -1 to 1
    
    return np array with x_train, x_test, y_train, y_test
    """
    
    idx = next(os.walk('/data/jimmy15923/cg_kidney_seg/train'))[1]
    # remove two file with different size between image & mask
    idx.remove("S2016-30816_9_0")
    idx.remove("S2016-30816_9_1")
    
    # set seed
    random.seed(split_seed)
    random.shuffle(idx)
    
    train_idx, test_idx = idx[:int(len(idx)*train_ratio)], idx[int(len(idx)*train_ratio):]

    x_train = np.array([cv2.imread('/data/jimmy15923/cg_kidney_seg/train/{}/image/{}_slide.jpg'.format(x, x))[...,::-1]\
                    for x in train_idx], dtype="float32")
    x_test = np.array([cv2.imread('/data/jimmy15923/cg_kidney_seg/train/{}/image/{}_slide.jpg'.format(x, x))[...,::-1]\
                       for x in test_idx], dtype="float32")
    
    if is_normalize:
        x_train = (x_train / 127.5) - 1
        x_test = (x_test / 127.5) - 1
        
    y_train = np.array([cv2.imread('/data/jimmy15923/cg_kidney_seg/train/{}/mask/{}_mask.jpg'.format(x, x))[..., 0]\
                    for x in train_idx])
    
    y_test = np.array([cv2.imread('/data/jimmy15923/cg_kidney_seg/train/{}/mask/{}_mask.jpg'.format(x, x))[..., 0]\
                        for x in test_idx])
    
    y_train = y_train.astype(np.bool)*1
    y_test = y_test.astype(np.bool)*1
    
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = read_data_and_split(split_seed=7, train_ratio=0.8, is_normalize=True)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
idx = np.random.choice(len(y_train))
plt.subplot(121)
plt.imshow(y_train[idx])
plt.subplot(122)
tmp = y_train[idx].astype(np.bool) * 1
plt.imshow(tmp)

In [ ]:
y_train_inv = np.where(y_train, 0, 1)
y_train_ = np.zeros(shape=(len(y_train), 1000,1000, 2))
y_train_[:,:,:,0] = y_train
y_train_[:,:,:,1] = y_train_inv

y_test_inv = np.where(y_test, 0, 1)
y_test_ = np.zeros(shape=(len(y_test), 1000,1000, 2))
y_test_[:,:,:,0] = y_test
y_test_[:,:,:,1] = y_test_inv

In [ ]:
idx = np.random.choice(len(y_train))
plt.subplot(121)
plt.imshow(y_train_[idx][:,:,0])
plt.subplot(122)
plt.imshow(y_train_[idx][:,:,1])

In [ ]:
    if augmentation:
        import imgaug

        # Augmentors that are safe to apply to masks
        # Some, such as Affine, have settings that make them unsafe, so always
        # test your augmentation on masks
        MASK_AUGMENTERS = ["Sequential", "SomeOf", "OneOf", "Sometimes",
                           "Fliplr", "Flipud", "CropAndPad",
                           "Affine", "PiecewiseAffine"]

        def hook(images, augmenter, parents, default):
            """Determines which augmenters to apply to masks."""
            return (augmenter.__class__.__name__ in MASK_AUGMENTERS)

        # Store shapes before augmentation to compare
        image_shape = image.shape
        mask_shape = mask.shape
        # Make augmenters deterministic to apply similarly to images and masks
        det = augmentation.to_deterministic()
        image = det.augment_image(image)
        # Change mask to np.uint8 because imgaug doesn't support np.bool
        mask = det.augment_image(mask.astype(np.uint8),
                                 hooks=imgaug.HooksImages(activator=hook))

In [ ]:
def data_gen(x_train, y_train, crop_size, bz, augmentation=None):
    while True:
        x = []
        y = []
        img_idx = np.random.choice(range(len(y_train)), bz, replace=False)
        for idx in img_idx:
            
            x_mid = np.random.choice(range(1000-(crop_size)), 1)[0] 
            y_mid = np.random.choice(range(1000-(crop_size)), 1)[0]
#             tmp_x = np.pad(x_train[idx], 1, mode='constant')
#             tmp_y = np.pad(y_train[idx], 1, mode='constant')
            x_ = x_train[idx][(x_mid):(x_mid+(crop_size)), (y_mid):(y_mid+(crop_size))]
            y_ = y_train[idx][(x_mid):(x_mid+(crop_size)), (y_mid):(y_mid+(crop_size))]
            if augmentation:
                import imgaug
                # Augmentors that are safe to apply to masks
                # Some, such as Affine, have settings that make them unsafe, so always
                # test your augmentation on masks
                MASK_AUGMENTERS = ["Sequential", "SomeOf", "OneOf", "Sometimes",
                                   "Fliplr", "Flipud", "CropAndPad",
                                   "Affine", "PiecewiseAffine"]

                def hook(images, augmenter, parents, default):
                    """Determines which augmenters to apply to masks."""
                    return (augmenter.__class__.__name__ in MASK_AUGMENTERS)

                # Store shapes before augmentation to compare
                image_shape = image.shape
                mask_shape = mask.shape
                # Make augmenters deterministic to apply similarly to images and masks
                det = augmentation.to_deterministic()
                x = det.augment_image(x)
                # Change mask to np.uint8 because imgaug doesn't support np.bool
                y = det.augment_image(y.astype(np.uint8),
                                         hooks=imgaug.HooksImages(activator=hook))
            x.append(x_)
            y.append(y_)

        x_ = np.array(x)
        y_ = np.array(y)
        yield x_, y_

In [ ]:
def val_gen(x_test, y_test, crop_size=500, stride=250):
    i = 0
    while True:
        x = []
        y = []
        for x_start in range(0, crop_size+1, stride):
            for y_start in range(0, crop_size+1, stride):
                x_crop = x_test[i][x_start:(x_start+crop_size), y_start:(y_start+crop_size), :]
                y_crop = y_test[i][x_start:(x_start+crop_size), y_start:(y_start+crop_size), :]
                x.append(x_crop)
                y.append(y_crop)
        i+=1
        yield np.array(x), np.array(y)
        if i == len(y_test):
            i=0

In [ ]:
from math import ceil
def img_combine(img, ncols=5, size=1, path=False):
    nimg=len(img)
    nrows=int(ceil(nimg/ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True, figsize=(ncols*size,nrows*size))
    if nrows==0:
        return
    elif ncols == 1:
        for r, ax in zip(np.arange(nrows), axes):
            nth=r
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow')
            ax.set_axis_off()
    elif nrows==1:
        for c, ax in zip(np.arange(ncols), axes):
            nth=c
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow' )
            ax.set_axis_off()
    else:
        for r, row in zip(np.arange(nrows), axes):
            for c, ax in zip(np.arange(ncols), row):
                nth=r*ncols+c
                if nth < nimg:
                    ax.imshow(img[nth], cmap='rainbow')
                ax.set_axis_off()
    
    if path:
        plt.tight_layout()
        plt.savefig(path, dpi = 300)
    plt.show()

In [ ]:
x, y = next(data_gen(x_train, y_train_, 500, 8))
print(x.shape)
print(y.shape)
plt.subplot(121)
plt.imshow(y[0][:,:,0])
plt.subplot(122)
plt.imshow(x[0].astype("uint8"))

In [ ]:
# model.fit(x_test_, np.expand_dims(y_test_, 3), batch_size=10,
#                     epochs=10)

In [ ]:
crop_size = 500
model = Deeplabv3(input_shape=(crop_size, crop_size, 3), classes=2, OS=8)
logits = model.output
output = keras.layers.Activation("softmax")(logits)
model = Model(model.input, output)
model.summary()

In [ ]:
def dice_coef_loss(y_true, y_pred, smooth = 1):
    def dice_coef_fix(y_true, y_pred):
        intersection = K.sum(K.abs(y_true * y_pred), axis = -1)
        iou = (2. * intersection + smooth) / (K.sum(K.square(y_true), -1) + K.sum(K.square(y_pred),-1) + smooth)
        return iou
    loss = 1 - dice_coef_fix(y_true, y_pred)
    return loss

In [ ]:
from keras import backend as K
import keras
from keras.utils import to_categorical

early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=12, verbose=1)
check = keras.callbacks.ModelCheckpoint(monitor="val_loss",
                                        filepath="/data/jimmy15923/cg_kidney_seg/test.h5",
                                        verbose=1, save_best_only=True)

reduce = keras.callbacks.ReduceLROnPlateau(patience=3),
model.compile(optimizer=keras.optimizers.SGD(lr=1e-4, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy')

vgen = val_gen(x_test, y_test_)
model.fit_generator(data_gen(x_train, y_train_, crop_size, 12),
                    steps_per_epoch=200,
                    epochs=10, 
                    validation_data=vgen,
                    validation_steps=len(y_test_)
                   )

In [ ]:
from deeplab_v3_plus.model import relu6, BilinearUpsampling
from keras.models import load_model
deeplab_model = load_model("/data/jimmy15923/cg_kidney_seg/test.h5",
                           custom_objects={'relu6':relu6,'BilinearUpsampling':BilinearUpsampling })

In [ ]:
model = Model(inputs=[model.input], outputs=[model.output])
model.load_weights('/data/jimmy15923/cg_kidney_seg/test.h5')

In [ ]:
vgen = val_gen(x_test, y_test_, crop_size=500, stride=500)

In [ ]:
x, y = next(data_gen(x_train, y_train_, crop_size, 2))

In [ ]:
x, y = next(vgen)
print(x.shape)
print(y.shape)
y_pred = model.predict(x)

In [ ]:
img_combine(x, ncols=2)
img_combine(y[:,:,:,0], ncols=2)
img_combine((y_pred[:,:,:,0] > 0.5)*1, ncols=2)

In [ ]:
idx = np.random.choice(len(y))
y_show = y_pred[idx][:,:,1]
y_show[y_show >= 0.5]*1
plt.subplot(121)
plt.imshow(y[idx][:,:,0])
plt.subplot(122)
plt.imshow(y_show.astype("uint8"))

In [ ]:
np.argmax(y_pred.squeeze(), -1).sum()

In [ ]:
y_pred.shape

In [ ]:
y_pred.squeeze().shape

In [ ]:
y_pred[0].squeeze().shape

In [ ]:
plt.imshow(np.argmax(y_pred[3].squeeze(), -1))